<a href="https://colab.research.google.com/github/Kennedy87670/NLP/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# introduction to NLP Fundamentals in Tensorflow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

# Check our GPU

In [1]:
!nvidia-smi - l

Invalid combination of input arguments. Please run 'nvidia-smi -h' for help.



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-08-26 10:01:31--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-26 10:01:31 (62.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we are going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labeled as diaster or not diaster).

See the original source here https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#unzip data
unzip_data('nlp_getting_started.zip')

--2022-08-26 10:01:34--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 142.251.10.128, 142.251.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-08-26 10:01:34 (144 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



# Visualize a text dataset


In [4]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df =pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# what does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
#how many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# how many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# lets visualize some random samples
import random
random_index = random.randint(0, len(train_df)-5) #create random indexes
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target:{target}", '(real diaster)' if target >0 else '(not real diaster' )
  print(f'Text:\n{text}\n')
  print('---\n')

Target:1 (real diaster)
Text:
Police Officer Wounded Suspect Dead After Exchanging Shots: Richmond police officer wounded suspect killed a... http://t.co/5uFTRXPpV0

---

Target:0 (not real diaster
Text:
act my age was a MESS everyone was so wild it was so fun my videos a wreck

---

Target:0 (not real diaster
Text:
A true #TBT  Eyewitness News WBRE WYOU http://t.co/JHVigsX5Jg

---

Target:1 (real diaster)
Text:
LLF TALK  WORLD NEWS U.S. in record hurricane drought - The United States hasn't been hit by a major hurricane in ... http://t.co/ML8IrhWg7O

---

Target:0 (not real diaster
Text:
USW: 'The damage from abandoning the deal could well create a new level of uncertainty...economic upheaval &amp; military unrest'

---



In [10]:
## split data into training and validation dataset
from sklearn.model_selection import train_test_split

In [11]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42)

In [12]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)


(6851, 6851, 762, 762)

In [13]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers
 
when dealing with text problem, one of the first things you will have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vecto for each token (the size of the feature vetor can be defined and this embedding can be learned)

## Text Vectorization (tokenization)

In [14]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None


In [16]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
# setup text vectorization variable
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length= 15  # max length our sequences will be (e.g how many words from a tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [18]:
# fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)


In [19]:
# create a sample sentence and tokenize it
sample_sentence = "Theres a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# choose a random sentence from the training datset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"original text: \n {random_sentence}\
\n\nVectorized version: ")
text_vectorizer([random_sentence])

original text: 
 We should all have a fire safety plan. RT @Matt_Kroschel: MOCK WILDFIRE near #Vail as agencies prepare for the worst. http://t.co/SWwyLRk0fv

Vectorized version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  46,  168,   44,   24,    3,   42,  992,  241,   96,    1, 5103,
         146,  217, 7091,   26]])>

In [21]:
# get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()  #get all the unique words in our training data 
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"NMumber of words in vocab: {len(words_in_vocab)}")
print(f" 5 most common words:{top_5_words} ")
print(f" 5 most common words:{bottom_5_words} ")

NMumber of words in vocab: 10000
 5 most common words:['', '[UNK]', 'the', 'a', 'in'] 
 5 most common words:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'] 


## Creatign an Embedding using Embedding layer

To make our embedding, we are going to use Tensorflow's embedding layers: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:
* Input dim = the size of our get_vocabulary
* output dim = the size of the output embedding vector, for example, a value of 100 would mean token get represented by a vector 100 long
* input_length = length of sequences passed to the embedding layers

In [22]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             embeddings_initializer='uniform',
                             output_dim=128,
                             input_length= max_length #how long each input
                             )
embedding

In [23]:
# Get a random sentence from the training set
random_sentence =random.choice(train_sentences)
print(f"Original text:\n {random_sentence} ")

# embedd the random sentence (turn it into dense vectors of fixed size)
sample_embed= embedding(text_vectorizer([random_sentence]))

sample_embed

Original text:
 [Tropical Cyclone Info] SOUDELOR 945hPa maximum wind speed: 45m/s maximum wind gust speed: 60m/s  http://t.co/nBD5oT9iEW 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03420975, -0.04737886,  0.00189735, ...,  0.04765585,
         -0.03179035,  0.00647758],
        [-0.00888718, -0.02847798,  0.01430986, ..., -0.01487981,
          0.0133461 ,  0.04521397],
        [-0.01646962, -0.00431839,  0.03705174, ..., -0.00436833,
         -0.04922065,  0.02114368],
        ...,
        [-0.01683618, -0.03893131,  0.0112058 , ..., -0.04451314,
         -0.03241408,  0.04221323],
        [ 0.02466596, -0.02187557,  0.01703575, ..., -0.02791109,
          0.0340369 ,  0.02928318],
        [ 0.02466596, -0.02187557,  0.01703575, ..., -0.02791109,
          0.0340369 ,  0.02928318]]], dtype=float32)>

In [24]:
# Check out a single token embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03420975, -0.04737886,  0.00189735, -0.03742235, -0.03983797,
        -0.03426363,  0.00286393, -0.0423921 , -0.03959794,  0.00356827,
         0.03109862,  0.0023975 , -0.04453143, -0.03207431, -0.02481666,
         0.0141902 ,  0.01912091, -0.02682496,  0.0300274 , -0.00139483,
        -0.04402082, -0.03302633, -0.03692285,  0.01956638, -0.02182032,
        -0.00451499,  0.00931959, -0.02564207,  0.02699671,  0.0422187 ,
         0.02277191,  0.0363921 ,  0.01884765, -0.03705461,  0.02128288,
        -0.03014014, -0.04058636, -0.00325596,  0.00365828,  0.01632107,
        -0.04831591,  0.03995943,  0.04359675, -0.0041814 , -0.02768373,
        -0.04497017, -0.01761606,  0.01526692,  0.01481313, -0.04072239,
         0.04883521,  0.03080758, -0.0143675 , -0.00623437,  0.04481244,
        -0.02727997, -0.04560719,  0.03022263, -0.01617559, -0.03356989,
         0.04274089, -0.00596193, -0.01339404,  0.02793949, -0.00342415,
  

# Modelling a text dataset (running a series of experiments)

Now we have got a way tot turn our text sequences into numbers, its time to start buildinjg a series of modelling experiments.

We will start witha  baseline and move on from there.

* Model 0: naive bayes (baseline), this is from sklearn ML map
* Model 1: Feed-forward neaural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* model 5: 1D convolutional Neautral network (CNN)
* model 6: Tensorflow hub Pretrained Feature Extractor (using transfer learning for NLP)
* mODEL 7: SAME AS MODEL 6 WITH 10% OF TRAINING DAta

how are we going to approach all these?

use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate our modelm

## model 0 : getting a baseline

As with all machine learning modelling experiments, its important to create a baseline model so you have got a benchmark for future experiments to build upon.
To create a baseline, we will use sklearn multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers

🔯 Note: its common pratice to use non DL algorithms as baseline of their speed and then later using DL to see if you can improve upon them.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# create a tokenization and modelling pipeline
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),  #convert words to numbers using tfid
                    ('clf', MultinomialNB()) # model the text
    
])

# fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

our baseline model achieves an accuracy of: 79.27%


In [27]:
# make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

Creating an evaluation function for our model experiments
We could evaluate these as they are but since we're going to be evaluating several models in the same way going forward, let's create a helper function which takes an array of predictions and ground truth labels and computes the following:

* Accuracy
* Precision
* Recall
* F1-score

🔑 Note: Since we're dealing with a classification problem, the above metrics are the most appropriate. If we were working with a regression problem, other metrics such as MAE (mean absolute error) would be a better choice.

In [28]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [29]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: A simple dense model
The first "deep" model we're going to build is a single layer dense model. In fact, it's barely going to have a single layer.

It'll take our text and labels as input, tokenize the text, create an embedding, find the average of the embedding (using Global Average Pooling) and then pass the average through a fully connected layer with one output unit and a sigmoid activation function.

If the previous sentence sounds like a mouthful, it'll make sense when we code it out (remember, if in doubt, code it out).

And since we're going to be building a number of TensorFlow deep learning models, we'll import our create_tensorboard_callback() function from helper_functions.py to keep track of the results of each.

In [30]:
# Create a tensorboard call back (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# create a directory to save TensorBoard logs
SAVE_DIR = 'model_logs'

In [31]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model


In [32]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [33]:
# Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [34]:
# Fit the model
model_1_history = model_1.fit(x= train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              y= train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20220826-100139
Epoch 1/5
215/215 [==============================] - 5s 5ms/step - loss: 0.6121 - accuracy: 0.6939 - val_loss: 0.5353 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4425 - accuracy: 0.8159 - val_loss: 0.4688 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3470 - accuracy: 0.8612 - val_loss: 0.4570 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2849 - accuracy: 0.8902 - val_loss: 0.4663 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2376 - accuracy: 0.9107 - val_loss: 0.4784 - val_accuracy: 0.7861


In [35]:
# check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4784 - accuracy: 0.7861


[0.47842419147491455, 0.7860892415046692]

In [36]:
# maqke some predictions and evaluate those
model_1_pred_probs= model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [37]:
# look at a single preiction
model_1_pred_probs[0]

array([0.38890946], dtype=float32)

In [38]:
# look at thr first 10 preiction
model_1_pred_probs[:10]

array([[0.38890946],
       [0.7808288 ],
       [0.99741167],
       [0.10486844],
       [0.10217419],
       [0.9280337 ],
       [0.91399974],
       [0.9923218 ],
       [0.96746206],
       [0.26294973]], dtype=float32)

In [39]:
# convert model predictions probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [40]:
#calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7899352475221245,
 'recall': 0.7860892388451444,
 'f1': 0.7834372696728737}

In [41]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [42]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualize Embedding

In [43]:
# Get the vocabulary from the text vectorization layers
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [44]:
# model_1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [45]:
# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learnt by 5 epochs)
embed_weights= model_1.get_layer('embedding').get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding dim (output dim of our embedding layer)

(10000, 128)


Now we've got the embedding matrix model has learned to represnt our tokens, lets see how we can visualize it.

To do so. Tensorflow has a handy tool called projector:
https://www.tensorflow.org/text/guide/word_embeddings

In [46]:
# create embedding files (we got this from tensorflow's word embedding documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()


In [47]:
# Download files from colab to upload to projector
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Network (RNN'S)
RNN's are useful for sequence data

The premise of a recurrent neural network is to use the represntation of a later night

## Model 2: LSTM

LSTM = long short term memory (one of the most popular LSTM cells)
Our structure of an RNN typically looks like this:

inputs(text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)

In [48]:
# Create an LSTM mOdel
from tensorflow.keras import layers


# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")


(None, 15, 128)
(None, 64)


In [49]:
# get a summary()
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [50]:

# compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [51]:
# Fit model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])


Saving TensorBoard log files to: model_logs/LSTM/20220826-100152
Epoch 1/5
215/215 [==============================] - 6s 8ms/step - loss: 0.2227 - accuracy: 0.9191 - val_loss: 0.5743 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1564 - accuracy: 0.9419 - val_loss: 0.6304 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1261 - accuracy: 0.9515 - val_loss: 0.7690 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1023 - accuracy: 0.9602 - val_loss: 0.7914 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0799 - accuracy: 0.9679 - val_loss: 1.0016 - val_accuracy: 0.7717


In [52]:
# make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[1.8209534e-03],
       [8.2489842e-01],
       [9.9980074e-01],
       [1.7700024e-02],
       [2.9222996e-04],
       [9.9477708e-01],
       [8.8141507e-01],
       [9.9976104e-01],
       [9.9968934e-01],
       [4.7944546e-01]], dtype=float32)

In [53]:

# convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [54]:

val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [55]:
# calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results


{'accuracy': 77.16535433070865,
 'precision': 0.7744533306963004,
 'recall': 0.7716535433070866,
 'f1': 0.7690403853056418}

## Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features  to an LSTM cell but has less parameters

In [56]:
from IPython.core.inputtransformer2 import leading_empty_lines
# build an RNN using GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
x = layers.GRU(64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each other, you need return sequence true
# print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x)
# print(x.shape)
# x = layers.GRU(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation='relu')(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

In [57]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 15, 64)            37248     
                                                                 
 global_average_pooling1d_1   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 1)                 

In [58]:
# Compile GRU model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [59]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20220826-100206
Epoch 1/5
215/215 [==============================] - 4s 9ms/step - loss: 0.1711 - accuracy: 0.9475 - val_loss: 0.6889 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1012 - accuracy: 0.9629 - val_loss: 0.7271 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0875 - accuracy: 0.9672 - val_loss: 0.8866 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0745 - accuracy: 0.9699 - val_loss: 1.0372 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0656 - accuracy: 0.9743 - val_loss: 0.9523 - val_accuracy: 0.7572


In [60]:
# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

((762, 1), array([[0.01634198],
        [0.92293966],
        [0.99988675],
        [0.08765721],
        [0.00141776],
        [0.99941385],
        [0.9445204 ],
        [0.9999944 ],
        [0.99998236],
        [0.4355171 ]], dtype=float32))

In [61]:

# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

((762, 1), array([[0.01634198],
        [0.92293966],
        [0.99988675],
        [0.08765721],
        [0.00141776],
        [0.99941385],
        [0.9445204 ],
        [0.9999944 ],
        [0.99998236],
        [0.4355171 ]], dtype=float32))

In [62]:
# Convert prediction probabilities to prediction classes
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [63]:
# Calcuate model_3 results
model_3_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 75.7217847769029,
 'precision': 0.758370695800474,
 'recall': 0.7572178477690289,
 'f1': 0.7550840315229969}

In [63]:
## Model_4 : bidirectional RNN

nORMAL rnnn'S GO FROM LEFT TO RIGHT (JUST LIKE YOU READ AN ENGLISH sentence)
